In [1]:
import os
import math
import numpy as np
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

from wsi.mapping.iso_name import ISO_NAME
from wsi.mapping.iso_gw import ISO_GW
from wsi.mapping.iso_iso2 import ISO_ISO2
from wsi.utils import raw_data_path, processed_data_path


In [2]:
# -----------------------------------------------------------------------------
# Population count functions (GPWv4 ASCII files)
# -----------------------------------------------------------------------------
EARTH_RADIUS_KM = 6371
# Fixed file pattern; note: files are for 2020 regardless of conflict year.
FILE_PATTERN = "gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_{tile}.asc"

def read_population_count(file_path):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return None, None, None
    with open(file_path, 'r') as f:
        metadata = {}
        for _ in range(6):
            parts = f.readline().strip().split()
            metadata[parts[0].lower()] = float(parts[1])
    ncols = int(metadata['ncols'])
    nrows = int(metadata['nrows'])
    xllcorner = metadata['xllcorner']
    yllcorner = metadata['yllcorner']
    cellsize = metadata['cellsize']
    nodata_value = metadata['nodata_value']
    data = np.loadtxt(file_path, skiprows=6)
    geotransform = (xllcorner, cellsize, 0, yllcorner + nrows*cellsize, 0, -cellsize)
    return data, geotransform, nodata_value

def bounding_box_degs(lat, lon, radius_km):
    dlat = radius_km / 111.0
    dlon = radius_km / (111.0 * math.cos(math.radians(lat)))
    return lat - dlat, lat + dlat, lon - dlon, lon + dlon

def latlon_to_rowcol(lat, lon, geotransform):
    origin_x, pixel_w, _, origin_y, _, pixel_h = geotransform
    col = int((lon - origin_x) / pixel_w)
    row = int((origin_y - lat) / -pixel_h)
    return row, col

def haversine_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return EARTH_RADIUS_KM * c

# -----------------------------------------------------------------------------
# Conflict data processing functions
# -----------------------------------------------------------------------------
def filter_conflicts(df, country_code, year):
    df = df[df['year'] == year]
    df = df[df['country_id'].astype(str).str.contains(str(country_code))]
    return df

def get_conflict_coordinates(df):
    return df[['latitude', 'longitude']].dropna().values.tolist()

# -----------------------------------------------------------------------------
# Sum population within conflict areas
# -----------------------------------------------------------------------------
def get_population_in_conflict_area(all_data, conflict_coords, radius_km=50):
    total_population = 0
    union_grid_points = []
    for dataset in all_data:
        data = dataset["data"]
        geotransform = dataset["geotransform"]
        no_data = dataset["no_data_value"]
        rows, cols = data.shape
        mask = np.zeros((rows, cols), dtype=bool)
        for (lat, lon) in conflict_coords:
            min_lat, max_lat, min_lon, max_lon = bounding_box_degs(lat, lon, radius_km)
            tl_row, tl_col = latlon_to_rowcol(max_lat, min_lon, geotransform)
            br_row, br_col = latlon_to_rowcol(min_lat, max_lon, geotransform)
            row_start = max(0, min(tl_row, br_row))
            row_end   = min(rows, max(tl_row, br_row) + 1)
            col_start = max(0, min(tl_col, br_col))
            col_end   = min(cols, max(tl_col, br_col) + 1)
            for r in range(row_start, row_end):
                for c in range(col_start, col_end):
                    origin_x, pixel_w, _, origin_y, _, pixel_h = geotransform
                    pixel_lat = origin_y + r * pixel_h
                    pixel_lon = origin_x + c * pixel_w
                    if haversine_distance(lat, lon, pixel_lat, pixel_lon) <= radius_km:
                        mask[r, c] = True
        valid_mask = mask & (data != no_data)
        pop_sum = data[valid_mask].sum()
        total_population += pop_sum
        for r in range(rows):
            for c in range(cols):
                if valid_mask[r, c]:
                    origin_x, pixel_w, _, origin_y, _, pixel_h = geotransform
                    pixel_lat = origin_y + r * pixel_h
                    pixel_lon = origin_x + c * pixel_w
                    union_grid_points.append([pixel_lat, pixel_lon, data[r, c]])
    return total_population, union_grid_points

# -----------------------------------------------------------------------------
# Clip grid points to a country polygon
# -----------------------------------------------------------------------------
def clip_grid_points_to_country(grid_points, country_polygon):
    clipped = []
    for pt in grid_points:
        point = Point(pt[1], pt[0])  # (lon, lat)
        if country_polygon.contains(point):
            clipped.append(pt)
    return clipped


# -----------------------------------------------------------------------------
# Main execution across multiple years using a fixed population raster (2020)
# -----------------------------------------------------------------------------

# Determine ISO codes.
# country_code_input = "700"  # e.g., "700" for Afghanistan
#valid_gw_codes = ["700", "900", "771", "760", "835", "811", "732", "731", "750", "850", "740", "812", "820", "781", "712", "775", "790", "920", "770", "710", "840", "830", "780","713", "800", "860", "816", "910", "950", "940"]

valid_gw_codes = ["811"]

# Define years to process.
years = [i for i in range(2000,2025)]
years = [2008, 2009, 2011]

# Load population count data once.
all_data = []
for tile in range(1, 9):
    file_pattern = FILE_PATTERN.format(tile=tile)
    file_path = raw_data_path("shocks", "gpw-v4", file_pattern)
    data, geotransform, no_data_value = read_population_count(file_path)
    if data is not None:
        all_data.append({
            "file": file_path,
            "data": data,
            "geotransform": geotransform,
            "no_data_value": no_data_value
        })
        print(f"Loaded population data: {file_path}")

# Read national population totals.
df_pop = pd.read_excel(raw_data_path("shocks", 'WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx'), 
                    sheet_name="Estimates", skiprows=16)
df_pop = df_pop[['ISO3 Alpha-code', 'Year', 'Total Population, as of 1 January (thousands)']].dropna()
df_pop['Year'] = df_pop['Year'].astype(int)
df_pop = df_pop.rename(columns={'ISO3 Alpha-code':'ISO_code', 
                                'Total Population, as of 1 January (thousands)': 'Population'})
df_pop['Population'] = df_pop['Population'] * 1000
df_pop = df_pop[df_pop['ISO_code'].isin(ISO_GW.keys())]

# Load conflict event CSVs.
UcdpPrioConflict_csv = pd.read_csv(raw_data_path("shocks", "UcdpPrioConflict_v24_1.csv"))
ids = UcdpPrioConflict_csv['conflict_id'].unique()
event_csv = pd.read_csv(raw_data_path("shocks", "GEDEvent_v24_1.csv"))
event_csv = event_csv[event_csv['conflict_new_id'].isin(ids)]

# Load country shapefile and reproject to WGS84.
countries = gpd.read_file(raw_data_path("shocks", "country_shapefiles", "World_Countries_Generalized.shp"))
countries = countries.to_crs("EPSG:4326")

country_yr_metrics = {}
for country_code_input in valid_gw_codes:
    selected_iso3 = None
    for iso3, code in ISO_GW.items():
        if str(code) == country_code_input:
            selected_iso3 = iso3
            break
    if selected_iso3 is None:
        print("Country code not found in ISO_GW.")
        continue
    selected_iso2 = ISO_ISO2[selected_iso3]

    country_gdf = countries[countries['ISO'] == selected_iso2]
    if country_gdf.empty:
        print("Country not found in shapefile.")
        continue
    country_polygon = country_gdf.geometry.iloc[0]
    country_name = country_gdf['COUNTRY'].iloc[0]

    # Containers for per-year metrics and heatmap data.
    year_metrics = {}
    heatmap_data_list = []
    conflict_markers = {}
    union_grid_points_by_year = {}

    for yr in years:
        print(f"Processing year {yr}...")
        conflict_df = filter_conflicts(event_csv, country_code_input, yr)
        conflict_df = conflict_df[conflict_df['best'] > 0]
        conflict_coords = get_conflict_coordinates(conflict_df)
        if not conflict_coords:
            union_grid_points = []
            total_pop_in_conflict_area = 0
        else:
            total_pop_in_conflict_area, union_grid_points = get_population_in_conflict_area(all_data, conflict_coords, radius_km=50)
            union_grid_points = clip_grid_points_to_country(union_grid_points, country_polygon)
            total_pop_in_conflict_area = sum(pt[2] for pt in union_grid_points)
        national_pop_row = df_pop[(df_pop['ISO_code'] == selected_iso3) & (df_pop['Year'] == yr)]
        if national_pop_row.empty:
            national_population = None
            print(f"No national population data for year {yr}.")
        else:
            national_population = national_pop_row['Population'].iloc[0]
        if national_population and national_population > 0:
            percentage = (total_pop_in_conflict_area / national_population) * 100
        else:
            percentage = None
        year_metrics[yr] = {
            "conflict_area_population": total_pop_in_conflict_area,
            "national_population": national_population,
            "percentage": percentage
        }
        heatmap_points = [[pt[0], pt[1], pt[2]] for pt in union_grid_points]
        heatmap_data_list.append(heatmap_points)
        conflict_markers[yr] = conflict_coords
        union_grid_points_by_year[yr] = union_grid_points

    country_yr_metrics[iso3] = year_metrics
    for yr, metrics in year_metrics.items():
        if metrics["percentage"] is not None:
            print(f"Year {yr}: {metrics['conflict_area_population']} people within 50km of conflict events, which is {metrics['percentage']:.2f}% of the national population.")
        else:
            print(f"Year {yr}: Insufficient national population data to compute percentage.")

# Determine map center.
    if any(heatmap_data_list):
        for pts in heatmap_data_list:
            if pts:
                avg_lat = sum(pt[0] for pt in pts) / len(pts)
                avg_lon = sum(pt[1] for pt in pts) / len(pts)
                break
    else:
        avg_lat, avg_lon = 0, 0




Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_1.asc
Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_2.asc
Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_3.asc
Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_4.asc
Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_5.asc
Loaded population data: C:\Users\kbuc0011\Documents\WSI\data\raw\shocks\gpw-v4\gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_30_sec_6.asc
Loaded pop

C:\Users\kbuc0011\AppData\Local\Temp\ipykernel_31504\3504527669.py:150: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  event_csv = pd.read_csv(raw_data_path("shocks", "GEDEvent_v24_1.csv"))


Processing year 2008...
Processing year 2009...
Processing year 2011...
Year 2008: 368434.98060400074 people within 50km of conflict events, which is 2.64% of the national population.
Year 2009: 368434.98060400074 people within 50km of conflict events, which is 2.60% of the national population.
Year 2011: 368434.98060400074 people within 50km of conflict events, which is 2.52% of the national population.


In [3]:
country_yr_metrics

{'KHM': {2008: {'conflict_area_population': np.float64(368434.98060400074),
   'national_population': 13942822.0,
   'percentage': np.float64(2.6424706605592525)},
  2009: {'conflict_area_population': np.float64(368434.98060400074),
   'national_population': 14164137.0,
   'percentage': np.float64(2.601181989442779)},
  2011: {'conflict_area_population': np.float64(368434.98060400074),
   'national_population': 14611969.0,
   'percentage': np.float64(2.521460185167384)}}}